# ライブラリinput

In [1]:
import sys
import pandas as pd
from collections import Counter
import string
import jctconv
import emoji
import re

print(sys.version)

3.8.12 (default, Oct 12 2021, 13:49:34) 
[GCC 7.5.0]


# データチェック

In [2]:
df = pd.read_csv("all_rakuma.csv")

In [3]:
df.head(3)

,date,item_url,title,price,brand,category,good,bad,normal,ship_days,comments,last_log_date,good_counts,prefecture,delivery_fee,seller_url,size,picture_url,description,details_dict
0,2019/10/08 23:39:51,https://item.fril.jp/055f4080a2a0ae275db6ba7be...,Epiphone LTD Korina Explorer Bass,40000,Epiphone,楽器 > ベース > エレキベース,109,0,0,1-2日後,NaN,2000-01-01,0,岐阜県,着払い,https://fril.jp/shop/42df208014145c438f96e01f5...,なし,https://img.fril.jp/img/259586636/l/734809353....,コリーナボディ ブラックピックガード ゴールドパーツの豪華なエクスプローラー コリーナ ...,"{'カテゴリ': '楽器›ベース›エレキベース', 'サイズ': 'なし', 'ブランド':..."
1,2019/10/08 23:39:51,https://item.fril.jp/896a1d0197d05cf8431c136d9...,TVB-100 (SUN),38000,Electric Bass(R),楽器 > ベース > エレキベース,8,0,0,1-2日後,"['TOKYO GUITARS', '本モデルを含め、TOKYO GUITARS工房にて製造...",2019/08/31,0,東京都,送料込,https://fril.jp/shop/bb898547c8e0369f5f6621cce...,なし,https://img.fril.jp/img/223532100/l/635334018....,Tokyo Guitars モデル名 TVB 100 SUN Tokyo Cust...,"{'カテゴリ': '楽器›ベース›エレキベース', 'サイズ': 'なし', '商品の状態'..."
2,2019/10/08 23:39:51,https://item.fril.jp/6bdb6b4dd0f6c518938b94ec2...,TRB-4000 (BL),42000,Electric Bass(R),楽器 > ベース > エレキベース,8,0,0,1-2日後,NaN,2000-01-01,0,東京都,送料込,https://fril.jp/shop/bb898547c8e0369f5f6621cce...,なし,https://img.fril.jp/img/223294381/l/634674588....,Tokyo Guitars モデル名 TRB 4000 BL Tokyo Cust...,"{'カテゴリ': '楽器›ベース›エレキベース', 'サイズ': 'なし', '商品の状態'..."


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122070 entries, 0 to 122069
Data columns (total 20 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   date           122070 non-null  object
 1   item_url       122070 non-null  object
 2   title          122070 non-null  object
 3   price          122070 non-null  int64 
 4   brand          122070 non-null  object
 5   category       122070 non-null  object
 6   good           122070 non-null  int64 
 7   bad            122070 non-null  int64 
 8   normal         122070 non-null  int64 
 9   ship_days      122070 non-null  object
 10  comments       11186 non-null   object
 11  last_log_date  122070 non-null  object
 12  good_counts    122070 non-null  int64 
 13  prefecture     122070 non-null  object
 14  delivery_fee   122070 non-null  object
 15  seller_url     122070 non-null  object
 16  size           122070 non-null  object
 17  picture_url    122070 non-null  object
 18  desc

In [5]:
len(df)

122070

In [6]:
df.category.nunique()

1012

In [7]:
df.category.value_counts() / len(df) * 100

メンズ > 靴/シューズ > スニーカー                 9.174244
エンタメ/ホビー > おもちゃ/ぬいぐるみ > キャラクターグッズ    8.975178
エンタメ/ホビー > フィギュア > アニメ/ゲーム           5.418203
エンタメ/ホビー > CD > ポップス/ロック(洋楽)         2.919636
エンタメ/ホビー > フィギュア > 特撮                2.577210
                                       ...   
エンタメ/ホビー > 雑誌 > 絵本/児童書               0.000819
スマホ/家電/カメラ > 冷暖房/空調 > その他            0.000819
メンズ > ジャケット/アウター > ライダースジャケット        0.000819
楽器 > DTM/DAW > DAWソフトウェア             0.000819
エンタメ/ホビー > 漫画 > 女性漫画                 0.000819
Name: category, Length: 1012, dtype: float64

In [8]:
ctg_df = df.category.value_counts().reset_index()

In [9]:
over1000_data = ctg_df[ctg_df.category > 1000]

In [10]:
over1000_data.category.sum()

63006

In [11]:
over1000_data[['index']]

,index
0,メンズ > 靴/シューズ > スニーカー
1,エンタメ/ホビー > おもちゃ/ぬいぐるみ > キャラクターグッズ
2,エンタメ/ホビー > フィギュア > アニメ/ゲーム
3,エンタメ/ホビー > CD > ポップス/ロック(洋楽)
4,エンタメ/ホビー > フィギュア > 特撮
5,エンタメ/ホビー > 美術品/アンティーク > 絵画/タペストリー
6,スポーツ/アウトドア > フィッシング > ルアー用品
7,エンタメ/ホビー > トレーディングカード > シングルカード
8,エンタメ/ホビー > おもちゃ/ぬいぐるみ > ミニカー
9,エンタメ/ホビー > CD > アニメ


In [12]:
data = df.merge(over1000_data[['index']],left_on="category", right_on="index")

In [13]:
data.category.nunique()

22

# テキストの前処理を行う

In [14]:
data.title[:10]

0       テレカ 未使用品 仮面ライダー 生誕20周年 東映ビデオ販売 販促用非売品
1       送料無料 新品 DVD Perfume WORLD TOUR 1st 初回
2                      ☆ウルトラマン☆レーザーディスク ジャンク？
3         MAISHA （マイシャ）Sadao Watanabe　渡辺貞夫★LD
4    LD★PARKER'S MOOD（パーカーズ・ムード）LIVE AT BRAVA
5         レーザーディスク １０枚セット★ロスト・イン・スペース★タイタニック 
6                               レーザーディスク７点セット
7           LUPIN THE THIRD/ルパン三世　カリオストロの城　LD
8         【カラオケＬＤ】パイオニア レーザーカラオケ ヒット８ ２０５（帯付）
9         ★LD(レーザーディスク) WINGS OF LIGHT 松任谷由実★
Name: title, dtype: object

In [15]:
emoji.UNICODE_EMOJI.keys()

dict_keys(['en', 'es', 'pt', 'it', 'fr', 'de'])

In [16]:
d = emoji.UNICODE_EMOJI
emojis = "".join(d['en'].keys())

In [17]:
puncs = string.punctuation + "☆★◆■●【】『』「」"
puncs

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~☆★◆■●【】『』「」'

In [18]:
def han2zen(txt):
  txt = jctconv.h2z(txt, kana=True, digit=False, ascii=False)
  return jctconv.z2h(txt, kana=False, digit=True, ascii=True)

def remove_signs(txt):
  rm_signs = emojis+puncs
  for s in rm_signs:
    txt = txt.replace(s, " ")
  return txt

def clean_txt(txt):
  txt = han2zen(txt)
  txt = remove_signs(txt)
  txt_list = txt.upper().split()
  txt_list = [x for x in txt_list if len(x) > 1 and re.search(r"[亜-熙ぁ-んァ-ヶa-zA-Z]", x)]
  return list(Counter(txt_list))

In [19]:
t = " ".join(["nike","nike","shoes","2000","【ナイキ】「","ア嗚呼"])
print("t",t)
print(clean_txt(t))

t nike nike shoes 2000 【ナイキ】「 ア嗚呼
['NIKE', 'SHOES', 'ナイキ', 'ア嗚呼']


In [20]:
data.tail().title.apply(clean_txt)

63001                                         [マリオテニス]
63002    [非売品, SWITCH, ポケットモンスター, ポケモンカードバトル, クリアファイル]
63003      [新品未開封, 大乱闘スマッシュブラザーズ, SPECIAL, クロスクリーナー付き]
63004      [新品未開封, 大乱闘スマッシュブラザーズ, SPECIAL, クロスクリーナー付き]
63005      [新品未開封, 大乱闘スマッシュブラザーズ, SPECIAL, クロスクリーナー付き]
Name: title, dtype: object

In [21]:
data_title_ = data.title.apply(clean_txt)

In [22]:
data_title_

0              [テレカ, 未使用品, 仮面ライダー, 周年, 東映ビデオ販売, 販促用非売品]
1         [送料無料, 新品, DVD, PERFUME, WORLD, TOUR, ST, 初回]
2                              [ウルトラマン, レーザーディスク, ジャンク]
3             [MAISHA, マイシャ, SADAO, WATANABE, 渡辺貞夫, LD]
4        [LD, PARKER, MOOD, パーカーズ・ムード, LIVE, AT, BRAVA]
                              ...                      
63001                                          [マリオテニス]
63002     [非売品, SWITCH, ポケットモンスター, ポケモンカードバトル, クリアファイル]
63003       [新品未開封, 大乱闘スマッシュブラザーズ, SPECIAL, クロスクリーナー付き]
63004       [新品未開封, 大乱闘スマッシュブラザーズ, SPECIAL, クロスクリーナー付き]
63005       [新品未開封, 大乱闘スマッシュブラザーズ, SPECIAL, クロスクリーナー付き]
Name: title, Length: 63006, dtype: object

# テキストを数値に置き換える

In [23]:
# sklearn

from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB


In [24]:
category2idx = {c:idx for idx, c in enumerate(data.category.unique())}
idx2category = {idx:c for idx, c in enumerate(data.category.unique())}

In [25]:
X = data.title
y = data.category.apply(lambda x: category2idx[x])

# データを学習させる

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [27]:
pipeline = Pipeline([
  ("bow", CountVectorizer()),
  ("fridf", TfidfTransformer()),
  ("classifier", RandomForestClassifier()),
  ])


In [28]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('bow', CountVectorizer()), ('fridf', TfidfTransformer()),
                ('classifier', RandomForestClassifier())])

In [29]:
pred = pipeline.predict(X_test)

# 精度を評価する

In [30]:
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

           0       0.88      0.64      0.74       406
           1       0.90      0.93      0.91      1189
           2       0.88      0.77      0.82       422
           3       0.88      0.83      0.85       699
           4       0.62      0.89      0.73      3577
           5       0.75      0.38      0.51       395
           6       0.89      0.82      0.85      2138
           7       0.85      0.79      0.82      1033
           8       0.93      0.88      0.91       460
           9       0.95      0.82      0.88       379
          10       0.96      0.83      0.89       871
          11       0.80      0.57      0.66       481
          12       0.89      0.73      0.81       430
          13       0.93      0.79      0.85       399
          14       0.99      0.95      0.97       535
          15       0.94      0.90      0.92       830
          16       0.99      0.79      0.88       396
          17       0.99    

In [31]:
idx2category[17]

'メンズ > 靴/シューズ > スニーカー'

In [32]:
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

           0       0.88      0.64      0.74       406
           1       0.90      0.93      0.91      1189
           2       0.88      0.77      0.82       422
           3       0.88      0.83      0.85       699
           4       0.62      0.89      0.73      3577
           5       0.75      0.38      0.51       395
           6       0.89      0.82      0.85      2138
           7       0.85      0.79      0.82      1033
           8       0.93      0.88      0.91       460
           9       0.95      0.82      0.88       379
          10       0.96      0.83      0.89       871
          11       0.80      0.57      0.66       481
          12       0.89      0.73      0.81       430
          13       0.93      0.79      0.85       399
          14       0.99      0.95      0.97       535
          15       0.94      0.90      0.92       830
          16       0.99      0.79      0.88       396
          17       0.99    

# 学習済みモデルを保存する

In [33]:
target = ["New balance 990 V4 IG 27.5cm ニューバランス"]
target = ["新品　オニツカタイガー 25.0cm"]
prediction = pipeline.predict(target)
idx2category[prediction[0]]

'メンズ > 靴/シューズ > スニーカー'

In [34]:
import pickle

In [35]:
with open("rdmf.pickle", mode="wb") as f:
  pickle.dump(pipeline, f)

In [36]:
ls

Python_Django_scikit_learnでAIアプリを開発.ipynb*  manage.py*
ai_app/                                            myvenv/
all_rakuma.csv*                                    nlp/
db.sqlite3*                                        rdmf.pickle*
idx2category.csv*                                  requirements.txt*


In [37]:
with open("rdmf.pickle", mode="rb") as ff:
  model = pickle.load(ff)


In [38]:
model.predict(target)

array([17])

In [39]:
pd.DataFrame([{"k":k,"v":v} for k,v in idx2category.items()]).to_csv("idx2category.csv", index=False)

In [40]:
{row.k:row.v for idx, row in pd.read_csv("idx2category.csv").iterrows()}

{0: 'エンタメ/ホビー > エンタメその他 > その他',
 1: 'エンタメ/ホビー > CD > ポップス/ロック(洋楽)',
 2: 'エンタメ/ホビー > CD > ポップス/ロック(邦楽)',
 3: 'エンタメ/ホビー > CD > アニメ',
 4: 'エンタメ/ホビー > おもちゃ/ぬいぐるみ > キャラクターグッズ',
 5: 'エンタメ/ホビー > アニメグッズ > その他',
 6: 'エンタメ/ホビー > フィギュア > アニメ/ゲーム',
 7: 'エンタメ/ホビー > フィギュア > 特撮',
 8: 'レディース > ワンピース > ひざ丈ワンピース',
 9: 'インテリア/住まい/日用品 > 文房具 > ペン/マーカー',
 10: 'スポーツ/アウトドア > フィッシング > ルアー用品',
 11: 'エンタメ/ホビー > おもちゃ/ぬいぐるみ > ぬいぐるみ',
 12: 'エンタメ/ホビー > おもちゃ/ぬいぐるみ > 模型/プラモデル',
 13: 'メンズ > トップス > Tシャツ/カットソー(半袖/袖なし)',
 14: 'スマホ/家電/カメラ > オーディオ機器 > ヘッドフォン/イヤフォン',
 15: 'エンタメ/ホビー > トレーディングカード > シングルカード',
 16: '楽器 > ギター > エレキギター',
 17: 'メンズ > 靴/シューズ > スニーカー',
 18: 'エンタメ/ホビー > 美術品/アンティーク > 絵画/タペストリー',
 19: 'エンタメ/ホビー > おもちゃ/ぬいぐるみ > ミニカー',
 20: 'エンタメ/ホビー > テレビゲーム > 家庭用ゲーム本体',
 21: 'エンタメ/ホビー > テレビゲーム > 家庭用ゲームソフト'}

In [41]:
{row.v:row.k for idx, row in pd.read_csv("idx2category.csv").iterrows()}

{'エンタメ/ホビー > エンタメその他 > その他': 0,
 'エンタメ/ホビー > CD > ポップス/ロック(洋楽)': 1,
 'エンタメ/ホビー > CD > ポップス/ロック(邦楽)': 2,
 'エンタメ/ホビー > CD > アニメ': 3,
 'エンタメ/ホビー > おもちゃ/ぬいぐるみ > キャラクターグッズ': 4,
 'エンタメ/ホビー > アニメグッズ > その他': 5,
 'エンタメ/ホビー > フィギュア > アニメ/ゲーム': 6,
 'エンタメ/ホビー > フィギュア > 特撮': 7,
 'レディース > ワンピース > ひざ丈ワンピース': 8,
 'インテリア/住まい/日用品 > 文房具 > ペン/マーカー': 9,
 'スポーツ/アウトドア > フィッシング > ルアー用品': 10,
 'エンタメ/ホビー > おもちゃ/ぬいぐるみ > ぬいぐるみ': 11,
 'エンタメ/ホビー > おもちゃ/ぬいぐるみ > 模型/プラモデル': 12,
 'メンズ > トップス > Tシャツ/カットソー(半袖/袖なし)': 13,
 'スマホ/家電/カメラ > オーディオ機器 > ヘッドフォン/イヤフォン': 14,
 'エンタメ/ホビー > トレーディングカード > シングルカード': 15,
 '楽器 > ギター > エレキギター': 16,
 'メンズ > 靴/シューズ > スニーカー': 17,
 'エンタメ/ホビー > 美術品/アンティーク > 絵画/タペストリー': 18,
 'エンタメ/ホビー > おもちゃ/ぬいぐるみ > ミニカー': 19,
 'エンタメ/ホビー > テレビゲーム > 家庭用ゲーム本体': 20,
 'エンタメ/ホビー > テレビゲーム > 家庭用ゲームソフト': 21}